In [216]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
#pip install textstat
#pip install --upgrade gensim
from pathlib import Path
import json_lines, json, copy
import numpy as np
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import gensim.downloader as api
import gensim
import operator
from nltk.util import ngrams 
import nltk
from nltk.corpus import stopwords
from nltk import download
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from gensim import corpora

import csv
from textstat.textstat import textstat
stop_words = stopwords.words('english')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Rafael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [114]:
raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
raw_instance_dir = Path("../clickbait_dataset")
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
raw_truth_dir = Path("../clickbait_dataset")

In [115]:
#open with json - raw_instance_train
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[0]

### Fields in instances.jsonl:
##{
#"id" : "<instance id>",
#"postTimestamp" : "<weekday> <month> <day> <hour>:<minute>:<second><time_offset> <year>",
#"postText" : ["<text of the tweet post with links removed>"],
#"postMedia" : ["<path to a file in the media archive>"],
#"targetTitle" : "<title of target news article>",
#"targetDescription": "<description tag of target news article>",
#"targetKeywords" : "<keywords tag of target news article>",
#"targetParagraphs" : ["<text of the i-th paragraph in the target article>"],
#"targetCaptions" : ["<caption of the i-th image in the target article>"]}
### Fields in truth.jsonl:
#{
#"id" : "<instance id>",
#"truthJudgments" : [<number in [0,1]>],
#"truthMean" : <number in [0,1]>,
#"truthMedian" : <number in [0,1]>,
#"truthMode" : <number in [0,1]>,
#truthClass" : "clickbait | no-clickbait"
#}

{'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
  "The guidelines

In [116]:
#open with json - raw_truth_train

raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))
raw_truth_train[7]


{'truthMedian': 0.33333334,
 'truthMean': 0.2,
 'truthMode': 0.33333334,
 'truthClass': 'no-clickbait',
 'truthJudgments': [0.33333334, 0.0, 0.33333334, 0.33333334, 0.0],
 'id': '7'}

In [117]:
#check the properties
print(type(raw_instance_train))    #type for whole data: list
print(type(raw_instance_train[0]))  #type for each element of the liast: dict
print(len(raw_instance_train))
print('Check Dict. Indexes for instance_train: ' + str(raw_instance_train[0].keys()))
print('Check Dict. Indexes for truth_train: ' + str(raw_truth_train[0].keys()))

<class 'list'>
<class 'dict'>
17581
Check Dict. Indexes for instance_train: dict_keys(['targetCaptions', 'targetDescription', 'postText', 'postTimestamp', 'postMedia', 'targetParagraphs', 'targetTitle', 'id', 'targetKeywords'])
Check Dict. Indexes for truth_train: dict_keys(['truthMedian', 'truthMean', 'truthMode', 'truthClass', 'truthJudgments', 'id'])


In [118]:
raw_instance_train = [json.loads(line) for line in open("../clickbait_dataset/instances_train.jsonl")]
raw_truth_train = [json.loads(line) for line in open("../clickbait_dataset/truth_train.jsonl")]

In [119]:
#separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_instance_train)):
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = copy.deepcopy(raw_instance_train[x])
        temp_dict.update(raw_truth_train[x])
        clickbait_train.append(temp_dict)
    else:
        non_clickbait_train.append(raw_instance_train[x])

In [120]:
truth_label = []
for x in range(0, len(raw_instance_train)):
    truth_label.append(raw_truth_train[x]['truthClass'])
len(truth_label)

17581

In [121]:
#clickbait_train
clickbait_train[0]

{'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'postMedia': [],
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns about the star goalkeeper.",
  "The

In [122]:
#non_clickbait_train
non_clickbait_train[0]

{'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
  "The guidelines

In [225]:
#Lexicon (Word) Count
targetTitleCountWords = []
for x in range(0, len(raw_instance_train)):
    targetTitleCountWords.append(textstat.lexicon_count(raw_instance_train[x]['targetTitle']))
targetTitleCountWords

[17,
 8,
 10,
 11,
 6,
 11,
 10,
 11,
 12,
 14,
 10,
 12,
 12,
 23,
 10,
 14,
 9,
 24,
 7,
 10,
 12,
 10,
 7,
 5,
 11,
 14,
 8,
 11,
 10,
 7,
 14,
 8,
 11,
 10,
 9,
 11,
 7,
 19,
 11,
 1,
 9,
 13,
 12,
 3,
 10,
 299,
 15,
 9,
 10,
 295,
 11,
 21,
 7,
 7,
 10,
 11,
 10,
 9,
 10,
 10,
 12,
 8,
 19,
 10,
 307,
 295,
 8,
 4,
 5,
 9,
 9,
 4,
 11,
 10,
 276,
 9,
 21,
 9,
 8,
 8,
 11,
 8,
 18,
 6,
 20,
 8,
 12,
 15,
 10,
 1,
 12,
 9,
 20,
 10,
 8,
 13,
 13,
 11,
 13,
 6,
 12,
 12,
 10,
 11,
 12,
 10,
 8,
 11,
 12,
 10,
 10,
 11,
 11,
 9,
 8,
 10,
 14,
 7,
 8,
 14,
 10,
 8,
 20,
 15,
 11,
 10,
 6,
 14,
 9,
 12,
 10,
 9,
 13,
 17,
 7,
 10,
 12,
 6,
 18,
 17,
 8,
 7,
 10,
 13,
 16,
 17,
 12,
 13,
 301,
 7,
 4,
 11,
 12,
 10,
 12,
 18,
 9,
 12,
 7,
 5,
 12,
 4,
 8,
 11,
 9,
 9,
 9,
 13,
 13,
 2,
 10,
 8,
 7,
 11,
 10,
 10,
 20,
 10,
 10,
 9,
 7,
 10,
 11,
 25,
 11,
 36,
 11,
 12,
 3,
 8,
 7,
 9,
 9,
 18,
 303,
 11,
 16,
 12,
 11,
 9,
 7,
 10,
 13,
 5,
 10,
 10,
 7,
 20,
 11,
 12,
 6,
 9,
 7,
 8,


In [124]:
#Grade Level reading difficulty
#Coleman-Liau index, Gunning FOG Formula, Linsear Write Formula
targetTitle_coleman_liau_index = []
targetTitle_gunning_fog = []
targetTitle_linsear_write_formula = []
for x in range(0, len(raw_instance_train)):
    targetTitle_coleman_liau_index.append(textstat.coleman_liau_index(raw_instance_train[x]['targetTitle']))
    targetTitle_gunning_fog.append(textstat.gunning_fog(raw_instance_train[x]['targetTitle']))
    targetTitle_linsear_write_formula.append(textstat.linsear_write_formula(raw_instance_train[x]['targetTitle']))


In [125]:
print(raw_instance_train[0]['targetTitle'])
textstat.coleman_liau_index(raw_instance_train[0]['targetTitle'])


Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


12.12

In [126]:
print(raw_instance_train[0]['targetTitle'])
textstat.gunning_fog(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


20.564705882352943

In [127]:
print(raw_instance_train[0]['targetTitle'])
textstat.linsear_write_formula(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


6.5

In [128]:
#clickbait_train, emoji
def pos_1gram(data, title):
    postag_words = word_tokenize(data.get(title)[0])
    return nltk.pos_tag(postag_words)

pos_tag_list = []
#for single_data in clickbait_train:
    #print(pos_2gram(single_data, "postText"))

In [129]:
pos_1gram(clickbait_train[0], "postText")

[('U.S.', 'NNP'),
 ('Soccer', 'NNP'),
 ('should', 'MD'),
 ('start', 'VB'),
 ('answering', 'VBG'),
 ('tough', 'JJ'),
 ('questions', 'NNS'),
 ('about', 'IN'),
 ('Hope', 'NNP'),
 ('Solo', 'NNP'),
 (',', ','),
 ('@', 'NNP'),
 ('eric_adelson', 'NN'),
 ('writes', 'NNS'),
 ('.', '.')]

In [130]:
#calculate tags in a data
def cal_pos_1grams_tag(data, title, tag):
    i = 0
    pos_list = []
    postag_words = word_tokenize(data.get(title)[0])
    pos_list = nltk.pos_tag(postag_words)
    for tags in pos_list:
        #print("tags: ", tags)
        if tags[1] == tag:
            i+=1
    return i   

In [131]:
cal_pos_1grams_tag(clickbait_train[0], "postText", "NNP")
cal_pos_1grams_tag_list = []
for x in range(0, len(raw_instance_train)):
    cal_pos_1grams_tag_list.append(cal_pos_1grams_tag(raw_instance_train[x], "postText", "NNP"))
cal_pos_1grams_tag_list

[1,
 3,
 5,
 2,
 0,
 0,
 2,
 0,
 4,
 4,
 1,
 1,
 2,
 4,
 1,
 9,
 4,
 0,
 0,
 3,
 4,
 0,
 3,
 4,
 4,
 0,
 5,
 1,
 1,
 1,
 1,
 0,
 0,
 5,
 1,
 2,
 0,
 10,
 3,
 2,
 1,
 7,
 2,
 2,
 3,
 1,
 2,
 2,
 4,
 1,
 1,
 2,
 3,
 2,
 2,
 2,
 0,
 3,
 2,
 4,
 7,
 0,
 0,
 1,
 8,
 2,
 10,
 0,
 4,
 4,
 0,
 3,
 1,
 1,
 1,
 5,
 6,
 1,
 1,
 5,
 0,
 3,
 1,
 1,
 4,
 5,
 3,
 3,
 2,
 4,
 1,
 3,
 4,
 3,
 6,
 3,
 2,
 4,
 1,
 2,
 1,
 3,
 6,
 4,
 2,
 2,
 3,
 8,
 3,
 11,
 4,
 2,
 5,
 1,
 1,
 7,
 4,
 4,
 1,
 1,
 3,
 0,
 1,
 8,
 3,
 0,
 2,
 2,
 3,
 2,
 6,
 3,
 1,
 3,
 2,
 2,
 5,
 8,
 5,
 1,
 4,
 2,
 6,
 0,
 1,
 2,
 1,
 6,
 3,
 1,
 3,
 4,
 2,
 1,
 3,
 2,
 3,
 2,
 5,
 8,
 1,
 0,
 7,
 2,
 4,
 3,
 4,
 3,
 7,
 13,
 5,
 0,
 3,
 6,
 1,
 2,
 4,
 2,
 2,
 5,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 8,
 2,
 3,
 2,
 7,
 2,
 2,
 4,
 2,
 1,
 9,
 2,
 1,
 3,
 2,
 3,
 4,
 0,
 3,
 1,
 0,
 2,
 0,
 5,
 7,
 4,
 2,
 0,
 0,
 4,
 0,
 5,
 2,
 3,
 3,
 2,
 3,
 10,
 2,
 7,
 5,
 3,
 3,
 8,
 3,
 7,
 3,
 8,
 3,
 2,
 1,
 2,
 0,
 2,
 5,
 1,
 0,
 0,
 4,
 0,
 0,

In [132]:
#yutarochan's code
def pos_2gram(title, p1, p2):
    pos_list = pos_tag(word_tokenize(title))
    #print("-1: ", pos_list[:-1])
    #print("1: ", pos_list[1:])
    #print("zip: ", list(zip(pos_list[:-1], pos_list[1:])))
    #print(list(zip(pos_list[:-1], pos_list[1:]))[0][1])
    #print(list(zip(pos_list[:-1], pos_list[1:]))[1][1])
    return sum(map(lambda x: x[0][1] == p1 and x[1][1] == p2, zip(pos_list[:-1], pos_list[1:])))

In [133]:
clickbait_train[0]['targetTitle']

'U.S. Soccer should start answering tough questions about Hope Solo'

In [134]:
pos_2gram(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [135]:
#revised pos_2grams
def pos_2gram_sum(title, p1, p2):
    pos_2g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2g_num += 1
    return pos_2g_num

In [136]:
pos_2gram_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_NNP.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP'))
pos_2gram_NNP_NNP

[0,
 2,
 2,
 1,
 0,
 2,
 5,
 0,
 3,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 1,
 4,
 4,
 3,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 8,
 3,
 0,
 0,
 32,
 0,
 3,
 3,
 30,
 0,
 0,
 0,
 3,
 0,
 4,
 6,
 0,
 4,
 0,
 6,
 3,
 0,
 5,
 32,
 34,
 5,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 77,
 2,
 2,
 5,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 5,
 2,
 0,
 7,
 3,
 1,
 1,
 1,
 0,
 4,
 2,
 4,
 0,
 0,
 4,
 7,
 0,
 1,
 1,
 4,
 3,
 6,
 6,
 1,
 0,
 2,
 4,
 0,
 1,
 9,
 1,
 0,
 3,
 7,
 0,
 0,
 3,
 1,
 5,
 0,
 5,
 6,
 4,
 4,
 1,
 2,
 0,
 0,
 5,
 5,
 1,
 3,
 0,
 1,
 2,
 7,
 7,
 7,
 2,
 0,
 3,
 28,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 4,
 1,
 0,
 3,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 4,
 3,
 1,
 0,
 3,
 3,
 4,
 2,
 1,
 9,
 1,
 0,
 0,
 4,
 2,
 0,
 5,
 2,
 0,
 3,
 6,
 34,
 1,
 0,
 0,
 4,
 5,
 4,
 3,
 3,
 1,
 4,
 0,
 4,
 1,
 0,
 2,
 0,
 2,
 1,
 0,
 5,
 0,
 0,
 5,
 1,
 44,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 4,
 1,
 0,
 0,
 4,
 0,
 2,
 3,
 6,
 0,
 3,
 31,
 3,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 1,

In [137]:
#check pos_2grams
def pos_2gram_check(title, p1, p2):
    pos_2gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2gram_list.append([pos_z[0], pos_z[1]])
    return pos_2gram_list

In [138]:
print(pos_2gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP'))

[[('U.S.', 'NNP'), ('Soccer', 'NNP')], [('Hope', 'NNP'), ('Solo', 'NNP')]]


In [139]:
#revised pos_3grams
def pos_3gram_sum(title, p1, p2, p3):
    pos_3g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3g_num += 1
    return pos_3g_num

In [229]:
pos_3gram_NNP_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_NNP_NNP_NNP.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP', 'NNP'))
len(pos_3gram_NNP_NNP_NNP)
print(pos_3gram_NNP_NNP_NNP)

[0, 1, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 12, 0, 1, 2, 9, 0, 0, 0, 1, 0, 2, 4, 0, 3, 0, 2, 2, 0, 3, 9, 14, 4, 0, 1, 0, 0, 0, 0, 1, 49, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 5, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 0, 0, 5, 0, 0, 1, 5, 0, 0, 1, 0, 4, 0, 2, 4, 2, 0, 0, 1, 0, 0, 3, 1, 0, 1, 0, 0, 0, 4, 4, 3, 1, 0, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 2, 1, 1, 0, 6, 0, 0, 0, 3, 1, 0, 3, 1, 0, 2, 3, 12, 0, 0, 0, 1, 3, 3, 2, 2, 0, 3, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 26, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 2, 3, 0, 1, 12, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 7, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 0, 4, 6, 2, 0, 0, 0, 0, 3, 0, 0, 16, 2, 1, 0, 0, 6, 0, 0, 0, 0, 1, 0, 0, 6, 2, 27, 0, 2, 0, 3, 0, 0, 1, 0, 0, 0, 0

In [230]:
#check pos_3grams
def pos_3gram_check(title, p1, p2, p3):
    pos_3gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3gram_list.append([pos_z[0], pos_z[1],pos_z[2]])
    return pos_3gram_list

In [142]:
print(pos_3gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

[]


In [143]:
target_para_list = []
for n in clickbait_train[0]['targetParagraphs']:
    target_para_ = doc_punc_parser(n)
    #print(type(target_para_list))
    #print(target_para_)
    target_para_list = target_para_list + target_para_
    #print(n)
    #print(nltk.pos_tag(n))

target_para_tags = nltk.pos_tag(target_para_list)

In [144]:
def count_tags (tags_list):
    tag_fd = nltk.FreqDist(tag for (word, tag) in tags_list)
    return (tag_fd.most_common())

In [145]:
print(count_tags(target_para_tags))

#for x in range(0, len(raw_instance_train)):


[('NN', 115), ('NNP', 90), ('DT', 88), ('IN', 87), ('JJ', 44), ('VBZ', 41), ('RB', 40), ('.', 31), ('VB', 30), (',', 27), ('NNS', 24), ('TO', 23), ('CC', 20), ('POS', 16), ('VBD', 15), ('VBG', 14), ('PRP', 13), ('VBN', 12), ('PRP$', 10), ("''", 10), ('``', 9), ('VBP', 6), ('MD', 6), ('JJR', 5), ('WP', 4), ('WRB', 3), (':', 3), ('CD', 3), ('EX', 3), ('JJS', 2), ('(', 2), ('WDT', 2), (')', 2), ('RBR', 2), ('RBS', 1), ('RP', 1), ('NNPS', 1)]


In [146]:
def findtags(tag_prefix, tagged_text):
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].most_common(5)) for tag in cfd.conditions())

In [147]:
findtags("NN",target_para_tags)

{'NNP': [('Solo', 15),
  ('U.S.', 11),
  ('Soccer', 10),
  ('Gulati', 5),
  ('Hope', 4)],
 'NN': [('violence', 6),
  ('lot', 4),
  ('incident', 3),
  ('explanation', 3),
  ('game', 3)],
 'NNS': [('women', 2),
  ('questions', 2),
  ('months', 2),
  ('concerns', 1),
  ('values', 1)],
 'NNPS': [('Americans', 1)]}

In [148]:
#Whether 'target Title' Start with 5W1H.
wh_start = []
for x in range(0, len(raw_instance_train)):
    wh_start.append(word_tokenize(raw_instance_train[x]['targetTitle'].lower())[0] in ['who', 'when', 'where', 'how', 'why','what'])
#Ethan's edit, I delete the [0] after raw_instance_train[x]['targetTitle']. It's a string already
wh_sample = lambda x: word_tokenize('What is it'.lower())[0] in ['who', 'when', 'where', 'how', 'why','what']
len(wh_start)

17581

In [149]:
#Foward reference: Whether 'target Title' has Demonstratives(this, that, those, these) 
hasDemonstrative = []
for y in range(0, len(raw_instance_train)):
    hasDemonstrative.append(sum(map(lambda x: str(x) in ['this', 'that', 'those','these'], 
                                     word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0)


In [150]:
#Foward reference: Whether 'target Title' has definite article('the')
hasDefinite = []
for y in range(0, len(raw_instance_train)):
    hasDefinite.append(sum(map(lambda x: str(x) == 'the', word_tokenize(raw_instance_train[y]
                                                                         ['targetTitle'].lower()))) > 0)


In [151]:
#Foward reference: Whether 'target Title' has 3rd person pronoun('he','she','his','her','him')
hasThirdPronoun = []
for y in range(0, len(raw_instance_train)):
    hasThirdPronoun.append(sum(map(lambda x: str(x) in ['he','she','his','her','him'], word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0)


In [154]:
#Foward reference: Whether 'target Title' Start with adverbs.
startsAdverb = []
for x in range(0, len(raw_instance_train)):
    startsAdverb.append(pos_tag(word_tokenize(raw_instance_train[x]['targetTitle']))[0][1] == 'ADV')
# print(pos_tag(word_tokenize(raw_instance_train[0]['targetTitle']))[0][1])
len(startsAdverb)

17581

In [180]:
countLongestWord = []

for x in range(0, len(raw_instance_train)):
    countLongestWord.append(len(max(raw_instance_train[x]['targetTitle'].split(), key=len)))
countLongestWord

[10,
 8,
 9,
 6,
 7,
 8,
 11,
 12,
 9,
 8,
 10,
 9,
 12,
 13,
 8,
 8,
 11,
 9,
 12,
 10,
 11,
 7,
 9,
 11,
 10,
 6,
 9,
 7,
 6,
 7,
 9,
 6,
 10,
 7,
 13,
 7,
 9,
 8,
 10,
 4,
 12,
 8,
 8,
 9,
 10,
 18,
 12,
 9,
 11,
 18,
 9,
 13,
 10,
 12,
 8,
 9,
 10,
 7,
 13,
 10,
 10,
 8,
 9,
 7,
 18,
 18,
 14,
 4,
 7,
 7,
 9,
 8,
 11,
 8,
 13,
 7,
 11,
 11,
 8,
 5,
 10,
 8,
 10,
 11,
 12,
 7,
 13,
 9,
 9,
 6,
 8,
 7,
 12,
 9,
 12,
 8,
 8,
 8,
 9,
 10,
 10,
 9,
 9,
 10,
 8,
 9,
 7,
 7,
 11,
 10,
 11,
 11,
 9,
 9,
 12,
 9,
 15,
 7,
 12,
 9,
 11,
 10,
 8,
 9,
 8,
 9,
 14,
 10,
 14,
 8,
 9,
 13,
 9,
 8,
 11,
 10,
 9,
 12,
 10,
 11,
 11,
 9,
 11,
 11,
 11,
 10,
 7,
 9,
 18,
 8,
 8,
 8,
 11,
 10,
 9,
 11,
 9,
 12,
 8,
 18,
 11,
 8,
 9,
 7,
 8,
 8,
 8,
 10,
 11,
 7,
 11,
 15,
 6,
 10,
 7,
 11,
 8,
 10,
 10,
 7,
 11,
 9,
 9,
 10,
 10,
 12,
 12,
 10,
 9,
 9,
 5,
 11,
 9,
 10,
 18,
 10,
 10,
 8,
 8,
 11,
 7,
 9,
 9,
 13,
 11,
 10,
 10,
 12,
 6,
 11,
 6,
 17,
 10,
 7,
 9,
 11,
 12,
 10,
 7,
 18,
 9,
 7,
 12,


In [185]:
averageLengthWords = []

for x in range(0, len(raw_instance_train)):
    averageLengthWords.append(sum(len(word) for word in raw_instance_train[x]['targetTitle'].split()) / len(raw_instance_train[x]['targetTitle'].split()))
averageLengthWords

[5.117647058823529,
 5.875,
 5.7,
 4.7272727272727275,
 4.833333333333333,
 4.909090909090909,
 6.7,
 5.090909090909091,
 4.0,
 4.642857142857143,
 6.2,
 5.166666666666667,
 5.5,
 5.695652173913044,
 5.7,
 4.071428571428571,
 6.555555555555555,
 4.916666666666667,
 6.0,
 4.7,
 5.333333333333333,
 4.8,
 5.625,
 5.8,
 5.2727272727272725,
 3.642857142857143,
 6.875,
 4.0,
 3.8,
 4.857142857142857,
 4.571428571428571,
 4.25,
 6.090909090909091,
 3.909090909090909,
 4.818181818181818,
 4.818181818181818,
 6.142857142857143,
 4.947368421052632,
 4.818181818181818,
 4.0,
 4.555555555555555,
 5.076923076923077,
 4.5,
 6.25,
 6.2,
 5.4366666666666665,
 5.1875,
 4.888888888888889,
 5.1,
 5.378378378378378,
 4.7272727272727275,
 6.181818181818182,
 6.857142857142857,
 7.0,
 4.4,
 5.2727272727272725,
 4.6,
 5.0,
 6.2,
 5.5,
 5.166666666666667,
 4.625,
 4.9,
 4.0,
 5.373376623376624,
 5.405405405405405,
 6.625,
 3.0,
 3.6666666666666665,
 4.555555555555555,
 5.666666666666667,
 5.25,
 6.63636363636

In [189]:
#Foward reference: Whether 'target Title' Start with Punctuation.
startsPunctuation = []
for x in range(0, len(raw_instance_train)):
    startsPunctuation.append(raw_instance_train[0]['targetTitle'][0].isdigit())
startsPunctuation

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [191]:
pos_2gram_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_VBZ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'VBZ'))
pos_2gram_NNP_VBZ

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 8,
 0,
 1,
 0,
 8,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 7,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [192]:
pos_2gram_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_VBZ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'VBZ'))
pos_2gram_NNP_VBZ

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 8,
 0,
 1,
 0,
 8,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 7,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [197]:
pos_2gram_NN_IN = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NN_IN.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NN', 'IN'))
pos_2gram_NN_IN

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 12,
 2,
 0,
 0,
 12,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 12,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 7,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 14,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 13,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 0,
 0,


In [198]:
pos_2gram_PRP_VBP = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_PRP_VBP.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'PRP', 'VBP'))
pos_2gram_PRP_VBP

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,


In [202]:
title_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    title_LenCharacter.append(len(raw_instance_train[x]['targetTitle']))
title_LenCharacter

[103,
 54,
 66,
 62,
 34,
 64,
 76,
 67,
 64,
 79,
 71,
 73,
 77,
 153,
 66,
 70,
 67,
 141,
 48,
 56,
 75,
 57,
 52,
 33,
 68,
 64,
 63,
 54,
 47,
 40,
 77,
 41,
 77,
 53,
 63,
 63,
 49,
 112,
 63,
 4,
 49,
 78,
 65,
 29,
 72,
 1930,
 98,
 53,
 61,
 1887,
 63,
 157,
 54,
 56,
 53,
 68,
 55,
 53,
 71,
 64,
 73,
 44,
 118,
 49,
 1962,
 1895,
 61,
 16,
 27,
 49,
 60,
 25,
 83,
 57,
 1766,
 51,
 139,
 63,
 40,
 39,
 83,
 46,
 107,
 43,
 138,
 44,
 71,
 100,
 50,
 6,
 83,
 48,
 124,
 59,
 51,
 75,
 79,
 63,
 73,
 39,
 73,
 61,
 68,
 70,
 68,
 62,
 45,
 57,
 61,
 64,
 76,
 65,
 67,
 57,
 59,
 71,
 100,
 35,
 65,
 73,
 64,
 57,
 109,
 94,
 64,
 54,
 46,
 79,
 76,
 61,
 61,
 65,
 85,
 112,
 50,
 67,
 73,
 56,
 113,
 107,
 54,
 46,
 76,
 90,
 86,
 90,
 58,
 85,
 1921,
 39,
 25,
 68,
 72,
 70,
 79,
 106,
 52,
 77,
 42,
 39,
 80,
 32,
 51,
 62,
 64,
 56,
 51,
 77,
 77,
 13,
 70,
 64,
 37,
 68,
 45,
 73,
 101,
 49,
 62,
 50,
 45,
 71,
 77,
 151,
 67,
 201,
 74,
 71,
 23,
 60,
 30,
 61,
 51,
 103,

In [203]:
keyword_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    keyword_LenCharacter.append(len(raw_instance_train[x]['targetKeywords']))
keyword_LenCharacter

[78,
 129,
 0,
 132,
 70,
 0,
 66,
 0,
 161,
 0,
 47,
 43,
 108,
 21,
 86,
 19,
 55,
 94,
 58,
 0,
 45,
 0,
 0,
 70,
 242,
 0,
 191,
 0,
 0,
 0,
 0,
 0,
 122,
 60,
 0,
 66,
 194,
 32,
 0,
 0,
 71,
 29,
 0,
 0,
 0,
 109,
 0,
 161,
 92,
 108,
 0,
 132,
 0,
 330,
 13,
 0,
 0,
 77,
 0,
 88,
 61,
 76,
 63,
 0,
 249,
 117,
 117,
 43,
 34,
 107,
 0,
 165,
 83,
 0,
 88,
 96,
 114,
 156,
 94,
 61,
 107,
 0,
 88,
 86,
 50,
 0,
 142,
 139,
 4,
 0,
 0,
 0,
 86,
 102,
 0,
 26,
 0,
 0,
 363,
 80,
 23,
 0,
 0,
 0,
 43,
 0,
 158,
 29,
 0,
 0,
 69,
 31,
 73,
 0,
 76,
 58,
 0,
 0,
 43,
 0,
 0,
 30,
 0,
 0,
 102,
 0,
 64,
 0,
 18,
 0,
 152,
 108,
 81,
 21,
 118,
 43,
 32,
 0,
 27,
 21,
 79,
 138,
 103,
 0,
 116,
 50,
 103,
 0,
 102,
 59,
 29,
 106,
 0,
 76,
 238,
 77,
 0,
 108,
 0,
 0,
 0,
 217,
 0,
 0,
 0,
 0,
 147,
 60,
 93,
 0,
 81,
 112,
 0,
 0,
 0,
 128,
 52,
 0,
 0,
 121,
 0,
 45,
 133,
 127,
 0,
 31,
 70,
 122,
 30,
 70,
 116,
 0,
 0,
 17,
 86,
 45,
 0,
 93,
 0,
 0,
 160,
 350,
 78,
 6,
 103,
 43,

In [211]:
caption_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    caption_LenCharacter.append(len(raw_instance_train[x]['targetCaptions']))
caption_LenCharacter

[10,
 36,
 3,
 4,
 3,
 0,
 0,
 0,
 16,
 1,
 1,
 1,
 0,
 4,
 2,
 1,
 1,
 23,
 2,
 1,
 0,
 0,
 0,
 525,
 2,
 0,
 6,
 4,
 8,
 0,
 2,
 1,
 31,
 299,
 14,
 20,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 5,
 4,
 3,
 1,
 4,
 1,
 36,
 0,
 0,
 0,
 1,
 1,
 31,
 0,
 0,
 10,
 0,
 3,
 1,
 4,
 5,
 27,
 7,
 1,
 10,
 1,
 0,
 1,
 0,
 8,
 0,
 12,
 24,
 0,
 0,
 10,
 4,
 24,
 0,
 1,
 2,
 26,
 2,
 0,
 2,
 1,
 1,
 13,
 4,
 0,
 3,
 34,
 0,
 2,
 0,
 2,
 1,
 1,
 13,
 36,
 10,
 4,
 0,
 122,
 1,
 2,
 12,
 13,
 209,
 0,
 26,
 26,
 1,
 0,
 1,
 0,
 13,
 5,
 27,
 1,
 4,
 0,
 0,
 31,
 2,
 6,
 3,
 1,
 0,
 1,
 1,
 1,
 3,
 480,
 7,
 5,
 0,
 2,
 1,
 4,
 1,
 1,
 6,
 74,
 7,
 5,
 1,
 1,
 12,
 1,
 0,
 0,
 11,
 5,
 34,
 3,
 0,
 0,
 0,
 27,
 1,
 1,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 16,
 0,
 0,
 8,
 25,
 1,
 21,
 36,
 13,
 42,
 0,
 13,
 203,
 0,
 0,
 2,
 4,
 0,
 2,
 8,
 0,
 2,
 13,
 1,
 1,
 31,
 0,
 1,
 1,
 1,
 0,
 30,
 0,
 2,
 1,
 7,
 5,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 9,
 1,
 4,
 7,
 74,
 12,
 18,
 1,
 1,
 1,
 1,
 4,
 2,
 14,
 30,

In [215]:
description_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    description_LenCharacter.append(len(raw_instance_train[x]['targetDescription']))
description_LenCharacter

[184,
 166,
 153,
 114,
 125,
 0,
 154,
 171,
 140,
 157,
 134,
 160,
 36,
 187,
 76,
 100,
 128,
 192,
 129,
 133,
 81,
 127,
 43,
 91,
 147,
 0,
 173,
 0,
 0,
 106,
 146,
 107,
 138,
 161,
 0,
 125,
 152,
 173,
 301,
 156,
 122,
 183,
 140,
 0,
 257,
 105,
 123,
 172,
 138,
 37,
 195,
 180,
 117,
 189,
 166,
 131,
 127,
 88,
 112,
 231,
 175,
 34,
 148,
 302,
 150,
 85,
 125,
 90,
 111,
 60,
 248,
 130,
 146,
 175,
 84,
 101,
 175,
 156,
 135,
 151,
 0,
 204,
 184,
 147,
 158,
 127,
 112,
 163,
 173,
 0,
 112,
 104,
 179,
 134,
 59,
 481,
 133,
 309,
 208,
 153,
 222,
 301,
 0,
 209,
 90,
 96,
 255,
 111,
 0,
 301,
 155,
 136,
 152,
 359,
 147,
 143,
 0,
 29,
 225,
 0,
 132,
 36,
 132,
 152,
 79,
 0,
 51,
 0,
 232,
 0,
 186,
 156,
 191,
 113,
 123,
 124,
 194,
 134,
 145,
 225,
 152,
 151,
 164,
 442,
 156,
 115,
 85,
 167,
 78,
 130,
 89,
 161,
 0,
 138,
 155,
 211,
 247,
 36,
 198,
 0,
 269,
 155,
 143,
 261,
 148,
 124,
 170,
 146,
 0,
 0,
 125,
 121,
 186,
 212,
 56,
 129,
 170,


In [210]:
title_keyword_CharacterDiff = list((map(operator.sub, title_LenCharacter, keyword_LenCharacter)))
title_keyword_CharacterDiff

[25,
 -75,
 66,
 -70,
 -36,
 64,
 10,
 67,
 -97,
 79,
 24,
 30,
 -31,
 132,
 -20,
 51,
 12,
 47,
 -10,
 56,
 30,
 57,
 52,
 -37,
 -174,
 64,
 -128,
 54,
 47,
 40,
 77,
 41,
 -45,
 -7,
 63,
 -3,
 -145,
 80,
 63,
 4,
 -22,
 49,
 65,
 29,
 72,
 1821,
 98,
 -108,
 -31,
 1779,
 63,
 25,
 54,
 -274,
 40,
 68,
 55,
 -24,
 71,
 -24,
 12,
 -32,
 55,
 49,
 1713,
 1778,
 -56,
 -27,
 -7,
 -58,
 60,
 -140,
 0,
 57,
 1678,
 -45,
 25,
 -93,
 -54,
 -22,
 -24,
 46,
 19,
 -43,
 88,
 44,
 -71,
 -39,
 46,
 6,
 83,
 48,
 38,
 -43,
 51,
 49,
 79,
 63,
 -290,
 -41,
 50,
 61,
 68,
 70,
 25,
 62,
 -113,
 28,
 61,
 64,
 7,
 34,
 -6,
 57,
 -17,
 13,
 100,
 35,
 22,
 73,
 64,
 27,
 109,
 94,
 -38,
 54,
 -18,
 79,
 58,
 61,
 -91,
 -43,
 4,
 91,
 -68,
 24,
 41,
 56,
 86,
 86,
 -25,
 -92,
 -27,
 90,
 -30,
 40,
 -45,
 85,
 1819,
 -20,
 -4,
 -38,
 72,
 -6,
 -159,
 29,
 52,
 -31,
 42,
 39,
 80,
 -185,
 51,
 62,
 64,
 56,
 -96,
 17,
 -16,
 13,
 -11,
 -48,
 37,
 68,
 45,
 -55,
 49,
 49,
 62,
 -71,
 45,
 26,
 -56,
 24,
 6

In [212]:
title_caption_CharacterDiff = list((map(operator.sub, title_LenCharacter, caption_LenCharacter)))
title_caption_CharacterDiff

[93,
 18,
 63,
 58,
 31,
 64,
 76,
 67,
 48,
 78,
 70,
 72,
 77,
 149,
 64,
 69,
 66,
 118,
 46,
 55,
 75,
 57,
 52,
 -492,
 66,
 64,
 57,
 50,
 39,
 40,
 75,
 40,
 46,
 -246,
 49,
 43,
 49,
 111,
 63,
 4,
 49,
 78,
 65,
 29,
 71,
 1928,
 98,
 48,
 57,
 1884,
 62,
 153,
 53,
 20,
 53,
 68,
 55,
 52,
 70,
 33,
 73,
 44,
 108,
 49,
 1959,
 1894,
 57,
 11,
 0,
 42,
 59,
 15,
 82,
 57,
 1765,
 51,
 131,
 63,
 28,
 15,
 83,
 46,
 97,
 39,
 114,
 44,
 70,
 98,
 24,
 4,
 83,
 46,
 123,
 58,
 38,
 71,
 79,
 60,
 39,
 39,
 71,
 61,
 66,
 69,
 67,
 49,
 9,
 47,
 57,
 64,
 -46,
 64,
 65,
 45,
 46,
 -138,
 100,
 9,
 39,
 72,
 64,
 56,
 109,
 81,
 59,
 27,
 45,
 75,
 76,
 61,
 30,
 63,
 79,
 109,
 49,
 67,
 72,
 55,
 112,
 104,
 -426,
 39,
 71,
 90,
 84,
 89,
 54,
 84,
 1920,
 33,
 -49,
 61,
 67,
 69,
 78,
 94,
 51,
 77,
 42,
 28,
 75,
 -2,
 48,
 62,
 64,
 56,
 24,
 76,
 76,
 13,
 69,
 61,
 34,
 68,
 45,
 73,
 85,
 49,
 62,
 42,
 20,
 70,
 56,
 115,
 54,
 159,
 74,
 58,
 -180,
 60,
 30,
 59,
 47,
 

In [219]:
caption_title_CharacterRatio = list((map(operator.truediv, caption_LenCharacter, title_LenCharacter)))


[0.0970873786407767,
 0.6666666666666666,
 0.045454545454545456,
 0.06451612903225806,
 0.08823529411764706,
 0.0,
 0.0,
 0.0,
 0.25,
 0.012658227848101266,
 0.014084507042253521,
 0.0136986301369863,
 0.0,
 0.026143790849673203,
 0.030303030303030304,
 0.014285714285714285,
 0.014925373134328358,
 0.16312056737588654,
 0.041666666666666664,
 0.017857142857142856,
 0.0,
 0.0,
 0.0,
 15.909090909090908,
 0.029411764705882353,
 0.0,
 0.09523809523809523,
 0.07407407407407407,
 0.1702127659574468,
 0.0,
 0.025974025974025976,
 0.024390243902439025,
 0.4025974025974026,
 5.6415094339622645,
 0.2222222222222222,
 0.31746031746031744,
 0.0,
 0.008928571428571428,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.013888888888888888,
 0.0010362694300518134,
 0.0,
 0.09433962264150944,
 0.06557377049180328,
 0.001589825119236884,
 0.015873015873015872,
 0.025477707006369428,
 0.018518518518518517,
 0.6428571428571429,
 0.0,
 0.0,
 0.0,
 0.018867924528301886,
 0.014084507042253521,
 0.484375,
 0.0,
 0.0,
 

In [218]:
description_title_CharacterRatio = list((map(operator.truediv, description_LenCharacter, title_LenCharacter)))


[1.7864077669902914,
 3.074074074074074,
 2.3181818181818183,
 1.8387096774193548,
 3.676470588235294,
 0.0,
 2.026315789473684,
 2.5522388059701493,
 2.1875,
 1.9873417721518987,
 1.8873239436619718,
 2.191780821917808,
 0.4675324675324675,
 1.2222222222222223,
 1.1515151515151516,
 1.4285714285714286,
 1.9104477611940298,
 1.3617021276595744,
 2.6875,
 2.375,
 1.08,
 2.2280701754385963,
 0.8269230769230769,
 2.757575757575758,
 2.161764705882353,
 0.0,
 2.746031746031746,
 0.0,
 0.0,
 2.65,
 1.896103896103896,
 2.6097560975609757,
 1.7922077922077921,
 3.0377358490566038,
 0.0,
 1.9841269841269842,
 3.1020408163265305,
 1.5446428571428572,
 4.777777777777778,
 39.0,
 2.489795918367347,
 2.3461538461538463,
 2.1538461538461537,
 0.0,
 3.5694444444444446,
 0.054404145077720206,
 1.2551020408163265,
 3.2452830188679247,
 2.262295081967213,
 0.0196078431372549,
 3.0952380952380953,
 1.1464968152866242,
 2.1666666666666665,
 3.375,
 3.1320754716981134,
 1.9264705882352942,
 2.309090909090

In [235]:
pos_3gram_NNP_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_NNP_NNP_VBZ.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP', 'VBZ'))
len(pos_3gram_NNP_NNP_VBZ)
print(pos_3gram_NNP_NNP_VBZ)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [236]:
pos_3gram_IN_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_IN_NNP_NNP.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'IN','NNP', 'NNP'))
len(pos_3gram_IN_NNP_NNP)
print(pos_3gram_IN_NNP_NNP)

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 1, 8, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 3, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 1, 1, 4, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 

In [237]:
pos_2gram_IN_NN = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_IN_NN.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'IN', 'NN'))
pos_2gram_IN_NN

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,


In [239]:
pos_2gram_NNP_NNS = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_NNS.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNS'))
pos_2gram_NNP_NNS

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 4,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [240]:
pos_2gram_IN_JJ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_IN_JJ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'IN', 'JJ'))
pos_2gram_IN_JJ

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 0,
 0,
 6,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,


In [241]:
import csv
from itertools import zip_longest

d = [pos_2gram_IN_JJ, pos_2gram_NNP_NNS, pos_2gram_IN_NN, pos_3gram_IN_NNP_NNP, pos_3gram_NNP_NNP_VBZ, 
     description_title_CharacterRatio, caption_title_CharacterRatio, title_caption_CharacterDiff, title_keyword_CharacterDiff, 
     description_LenCharacter, caption_LenCharacter, keyword_LenCharacter, title_LenCharacter, pos_2gram_PRP_VBP, pos_2gram_NN_IN, 
     pos_2gram_NNP_VBZ, startsPunctuation, averageLengthWords, countLongestWord, startsAdverb, hasThirdPronoun, 
     hasDemonstrative, wh_start, pos_3gram_NNP_NNP_NNP, pos_2gram_NNP_NNP, cal_pos_1grams_tag_list, 
     targetTitle_linsear_write_formula, targetTitle_gunning_fog, targetTitle_coleman_liau_index, 
     targetTitleCountWords, truth_label
    ]
export_data = zip_longest(*d, fillvalue = '')
with open('newFeatures.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("pos_2gram_IN_JJ",
                   "pos_2gram_NNP_NNS", "pos_2gram_IN_NN", "pos_3gram_IN_NNP_NNP", "pos_3gram_NNP_NNP_VBZ", "pos_3gram_NNP_NNP_VBZ",
                   "description_title_CharacterRatio", "caption_title_CharacterRatio", "title_caption_CharacterDiff", 
                   "title_keyword_CharacterDiff", "description_LenCharacter", "caption_LenCharacter", 
                   "keyword_LenCharacter", "title_LenCharacter", "startsPunctuation", "averageLengthWords", 
                   "countLongestWord", "startsAdverb", "hasThirdPronoun", "hasDemonstrative", "wh_start", 
                   "pos_3gram_NNP_NNP_NNP", "pos_2gram_NNP_NNP", "cal_pos_1grams_tag_list", 
                   "targetTitle_linsear_write_formula", "targetTitle_gunning_fog", "targetTitle_coleman_liau_index", 
                   "targetTitleCountWords","truth_label"))
      wr.writerows(export_data)
myfile.close()